In [ ]:
import numpy as np
import plotly.graph_objects as go
import plotly.express as px

from nemo.util import grid_2d
from nemo.plotting import plot_surface, plot_path_3d

%load_ext autoreload
%autoreload 2

g = 9.8  # m/s^2

Dynamics of a double integrator mass moving on a sloped plane

In [ ]:
slope = 0.1  # rad
x0 = np.array([0, 0])  # initial position
mass = 1.0  # kg

# Acceleration in heading angle theta (w.r.t. slope, theta=0 is directly uphill, theta=pi is directly downhill)
a = 1.0  # m/s^2
theta = 0  # rad

# Effective slope
slope_eff = slope * np.cos(theta)

net_F = mass * (a - g * np.sin(slope_eff))

Dynamics of a Dubin's car moving on a sloped plane (no slip)

- $\theta(t)$ - vehicle heading 
- $\alpha$ - slope angle
- $\psi$ - slope direction
- $\omega$ - yaw rate (commanded)
- $a$ - acceleration (commanded)
- $\alpha_{eff}$ - effective slope

$\theta(t) = \theta_0 + \omega t$ \
$\alpha_{eff}(t) = \alpha \cos(\theta(t) - \psi)$ \
$a_{net}(t) = a - g \sin(\alpha_{eff}(t))$

Full dynamics:
- State: $(x, y, V, \theta)$
    - Height field gives $z=H(x,y)$ and slope angle and direction thru gradient
- control: $(a, \omega)$
    - Fixed between waypoints

In [ ]:
slope = 0.1  # rad

theta_0 = 0  # rad
omega = 1.0  # rad/s

Load a height field

In [ ]:
import torch
from nemo.nemo import Nemo
from nemo.util import gradient

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

nemo = Nemo()
nemo.load_weights('../models/kt22_encs.pth', '../models/kt22_heightnet.pth')

In [ ]:
# nemo.plot()

Normal vector: (dh/dx, dh/dy, 1)


Coordinate frame:

Rover heading: \
+Y: North, $\theta = \pi/2$ \
+X: East, $\theta = 0$

Slope direction angle: \
+Y: $\psi = 0$ \
+X: $\psi = \pi/2$

positive gradient is uphill, negative gradient is downhill

In [ ]:
# Simulate dynamics for (a, omega)
init_state = np.array([-1., 0., 0., 0])  # (x, y, v, theta)
u = np.array([1.0, 0.0])     # (a, omega)
dt = 0.01

N_iters = 100
state = init_state
state_hist = []

for i in range(N_iters):
    # Unpack state and control
    x, y, v, theta = state
    a, omega = u

    # Query height field to get slope angle and direction
    pos = torch.tensor([[x, y]], device=device, requires_grad=True)
    _, grad = nemo.get_heights_with_grad(pos)
    psi = torch.arctan2(grad[0][1], grad[0][0])  # slope direction (0 is +Y, pi/2 is +X)
    slope = torch.arctan(grad.norm())  # slope angle (positive is uphill, negative is downhill) 
                                        # TODO: from tests below, it seems that NN grad slope is approximately
                                        #       x3 real slope for some reason - address/investigate this
    psi = psi.detach().cpu().numpy()
    slope = slope.detach().cpu().numpy()

    # Integrate theta
    theta += omega * dt  
    
    # Calculate effective slope
    slope_eff = slope * np.cos(theta - psi)
    
    # Calculate acceleration
    a_net = a - g * np.sin(slope_eff)

    # Integrate velocity and position
    v += a_net * dt
    x += v * np.cos(theta) * dt
    y += v * np.sin(theta) * dt

    # Update and log state
    state = np.array([x, y, v, theta])
    state_hist.append(state)

In [ ]:
state_hist = np.array(state_hist)
xy = state_hist[:,:2]

z_torch = nemo.get_heights(torch.tensor(xy, dtype=torch.float32).to(device))
z = z_torch.detach().cpu().numpy() + 1e-3

In [ ]:
fig = nemo.plot(N=128, bounds=(-1.0, 3.0, -1.0, 1.0))
fig = plot_path_3d(x=xy[:,0], y=xy[:,1], z=z.flatten(), fig=fig)
fig.show()

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter3d(x=xy[:,0], y=xy[:,1], z=z.flatten()))
fig.show()

In [ ]:
z[i]

In [ ]:
fig = go.Figure()
for i in range(N_iters):
    fig.add_trace(go.Scatter3d(x=[xy[i,0]], y=[xy[i,1]], z=[z.flatten()[i]]))
fig.show()

In [ ]:
N = 64
bounds = (-1, 1, -1, 1)
xs = torch.linspace(bounds[0], bounds[1], N, device=device)
ys = torch.linspace(bounds[2], bounds[3], N, device=device)
XY_grid = torch.meshgrid(xs, ys, indexing='xy')
XY_grid = torch.stack(XY_grid, dim=-1)
positions = XY_grid.reshape(-1, 2)
heights = nemo.get_heights(positions)

z_grid = heights.reshape(N, N).detach().cpu().numpy()
x_grid = XY_grid[:,:,0].detach().cpu().numpy()
y_grid = XY_grid[:,:,1].detach().cpu().numpy()

# fig = go.Figure()
# fig.add_trace(go.Surface(x=x_grid, y=y_grid, z=z_grid, colorscale='Viridis', showscale=False))
# fig.update_layout(width=1200, height=900, scene_aspectmode='data')

# fig.update_layout(
#          title='Animation Test',
#          width=1600,
#          height=900,
#          updatemenus=[dict(type='buttons',
#                   showactive=False,
#                   y=1,
#                   x=0.8,
#                   xanchor='left',
#                   yanchor='bottom',
#                   pad=dict(t=45, r=10),
#                   buttons=[dict(label='Play',
#                                  method='animate',
#                                  args=[None, dict(frame=dict(duration=5, redraw=True), 
#                                                              transition=dict(duration=0),
#                                                              fromcurrent=True,
#                                                              mode='immediate'
#                                                             )]
#                                             )
#                                       ]
#                               )
#                         ]
# )

frames=[]
for i in range(N_iters):
    frames.append(go.Frame(data=[go.Surface(x=x_grid, y=y_grid, z=z_grid, colorscale='Viridis', showscale=False), 
                                 go.Scatter3d(x=[xy[i,0]], y=[xy[i,1]], z=[z.flatten()[i]], 
                                 mode='markers', marker=dict(size=10, color='red'))]))

fig = go.Figure(
      data=[go.Surface(x=x_grid, y=y_grid, z=z_grid, colorscale='Viridis', showscale=False)],
      layout=go.Layout(
            updatemenus=[dict(type="buttons",
                              buttons=[dict(label="Play",
                                          method="animate",
                                          args=[None])])]),
      frames=frames
)
fig.update_layout(width=1200, height=900, scene_aspectmode='data')

fig.show()

Test slopes

In [ ]:
SLOPE = 1.0

N = 128
bounds = 2*np.array([-1., 1., -1., 1.])

xs = torch.linspace(bounds[0], bounds[1], N, device=device)
ys = torch.linspace(bounds[2], bounds[3], N, device=device)
XY_grid = torch.meshgrid(xs, ys, indexing='xy')
XY_grid = torch.stack(XY_grid, dim=-1)
x_grid = XY_grid[:,:,0].detach().cpu().numpy()
y_grid = XY_grid[:,:,1].detach().cpu().numpy()
xy_data = XY_grid.reshape(-1,2)

zs = torch.linspace(-SLOPE, SLOPE, N, device=device)
z_grid = zs.repeat(N,1)
z_data = z_grid.reshape(-1,1)
z_grid = z_grid.detach().cpu().numpy()

In [ ]:
fig = plot_surface(x=x_grid, y=y_grid, z=z_grid)
fig.show()

In [ ]:
import torch.nn as nn

In [ ]:
nemo = Nemo()
nemo.encoding.to(device)
nemo.height_net.to(device)

In [ ]:
xy_data = xy_data.half()
z_data = z_data.half()

In [ ]:
# Loss function
criterion = nn.MSELoss()

# Optimizer
optimizer = torch.optim.Adam([{'params': nemo.encoding.parameters()},
                              {'params': nemo.height_net.parameters()}], lr=1e-5)

# Train the network
for step in range(5000):
    # Forward pass
    pred = nemo.get_heights(xy_data)

    # Compute loss
    loss = criterion(pred, z_data)

    # Backward pass
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Print loss every 500 steps
    if step % 500 == 0:
        print(f"Step {step}, Loss {loss.item()}")

In [ ]:
z_pred = pred.reshape(N, N).detach().cpu().numpy()

fig = plot_surface(x=x_grid, y=y_grid, z=z_pred)
fig.show()

In [ ]:
xy_data.requires_grad = True
z, grad = nemo.get_heights_with_grad(xy_data)

In [ ]:
from plotly.subplots import make_subplots

In [ ]:
x_grad = grad[:,0].reshape(N,N).detach().cpu().numpy()
y_grad = grad[:,1].reshape(N,N).detach().cpu().numpy()

fig = make_subplots(rows=1, cols=2, subplot_titles=('X Gradient', 'Y Gradient'))
fig.add_trace(go.Heatmap(z=x_grad), row=1, col=1)
fig.add_trace(go.Heatmap(z=y_grad), row=1, col=2)
fig.update_layout(width=1200, height=600, scene_aspectmode='data')
fig.show()

In [ ]:
# Histogram of x_grad
fig = px.histogram(x_grad.flatten())
fig.show()

In [ ]:
fig = px.histogram(y_grad.flatten())
fig.show()